In [1]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

In [4]:
hn_dir = "../../out/201027/embedding/"

data = ["motor_toolkit","pfamA_motors_balanced","pfamA_random","pfamA_target"]
tuning_dir = ["evotune_balanced","evotune_balanced_target","mini_balanced","mini_balanced_target","raw"]
tsne_out_dir = "../../out/201027/tsne/"
pca_out_dir = "../../out/201027/pca/"

def joint_embed_tsne(target,background):
    jointed = np.concatenate((target, background), axis=0)
    pca_jointed = PCA(n_components=5).fit_transform(jointed)
    jointed_em_2 = TSNE(n_components=2).fit_transform(pca_jointed)
    target_em = jointed_em_2[0:target.shape[0],:]
    background_em = jointed_em_2[target.shape[0]:,:]
    return target_em,background_em

def joint_embed_pca(target,background):
    jointed = np.concatenate((target, background), axis=0)
    pca_jointed = PCA(n_components=2).fit_transform(jointed)
    target_em = pca_jointed[0:target.shape[0],:]
    background_em = pca_jointed[target.shape[0]:,:]
    return target_em,background_em

In [5]:
# # make one directory for each tuning method
# import os, sys
# for model_dir in models_dir:
#     os.mkdir(pca_out_dir+model_dir)
#     os.mkdir(tsne_out_dir+model_dir)
#     for tuning_dir_ in tuning_dir:
#         pca_path = pca_out_dir+model_dir+tuning_dir_
#         print(pca_path)
#         os.mkdir(pca_path)
#         tsne_path = tsne_out_dir+model_dir+tuning_dir_
#         os.mkdir(tsne_path);

## TSNE/PCA 
- directly PCA to 2 dimensions, or PCA to 10 then TSNE

In [8]:
import glob
# for model in models_dir:
import time 

model = models_dir[2]
print(model)
for tuning_dir_ in tuning_dir:
    print(tuning_dir_)
    dats = []
    for dat in data_dir:
        d_path = '../../out/201027/embedding/'+model+dat+'_'+tuning_dir_ + '.npy'
        print(d_path)
        d = np.load(d_path)
#         if dat == 'motor_toolkit':
#             d = d[motor_toolkit_short_ind,:]
        if dat == 'pfamA_target':
            d = d[pfamA_target_sub_ind,:]
        dats.append(d)
    print(dats[0].shape)
    print(dats[1].shape)
    print(dats[2].shape)
    print(dats[3].shape)
    # first, embed each of the dataset themselves
    pca_res = []
    tsne_res = []
    for i, dat in enumerate(dats):
        print(data_dir[i])
        start_time = time.time()
        pca_curr = PCA(n_components=2).fit_transform(dat)
        pca_res.append(pca_curr)
        tsne_curr = PCA(n_components=5).fit_transform(dat)
        tsne_curr = TSNE(n_components=2).fit_transform(tsne_curr)
        tsne_res.append(tsne_curr)
        print("--- %s seconds ---" % (time.time() - start_time))
        
    for i,dat_name in enumerate(data_dir):
        pca_out_path = pca_out_dir+model+tuning_dir_+"/"+dat_name+".npy"
        tsne_out_path = tsne_out_dir+model+tuning_dir_+"/"+dat_name+".npy"
        print(pca_out_path)
        print(tsne_out_path)
        np.save(pca_out_path, pca_res[i])
        np.save(tsne_out_path, tsne_res[i])
    # then, perform several jointly embedding
    #1. target,random
    #2. balanced, random
    #3. toolkit,target
    for i,j in [(0,2),(1,2)]:
        start_time = time.time()
        target_tsne,background_tsne = joint_embed_tsne(dats[i],dats[j])
        target_pca,background_pca = joint_embed_pca(dats[i],dats[j])
        print("--- %s seconds ---" % (time.time() - start_time))
        pca_out_path_t = pca_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_target.npy"
        tsne_out_path_t = tsne_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_target.npy"
        pca_out_path_b = pca_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_background.npy"
        tsne_out_path_b = tsne_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_background.npy"
        print(pca_out_path_t)
        print(pca_out_path_b)
        print(tsne_out_path_t)
        print(tsne_out_path_b)
        np.save(pca_out_path_t, target_pca)
        np.save(pca_out_path_b, background_pca)
        np.save(tsne_out_path_t, target_tsne)
        np.save(tsne_out_path_b, background_tsne)

transformer_encoder/
evotune_balanced
../../out/201027/embedding/transformer_encoder/motor_toolkit_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_motors_balanced_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_random_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_target_evotune_balanced.npy
(3235, 12)
(18000, 12)
(1600, 12)
(5544, 12)
motor_toolkit
--- 60.819462299346924 seconds ---
pfamA_motors_balanced
--- 377.482134103775 seconds ---
pfamA_random
--- 27.624279975891113 seconds ---
pfamA_target
--- 105.2143931388855 seconds ---
../../out/201027/pca/transformer_encoder/evotune_balanced/motor_toolkit.npy
../../out/201027/tsne/transformer_encoder/evotune_balanced/motor_toolkit.npy
../../out/201027/pca/transformer_encoder/evotune_balanced/pfamA_motors_balanced.npy
../../out/201027/tsne/transformer_encoder/evotune_balanced/pfamA_motors_balanced.npy
../../out/201027/pca/transformer_encoder/evotune_balanced/pfam

raw
../../out/201027/embedding/transformer_encoder/motor_toolkit_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_motors_balanced_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_random_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_target_raw.npy
(3235, 12)
(18000, 12)
(1600, 12)
(5544, 12)
motor_toolkit
--- 60.18629336357117 seconds ---
pfamA_motors_balanced
--- 363.3427255153656 seconds ---
pfamA_random
--- 26.91787075996399 seconds ---
pfamA_target
--- 105.67623782157898 seconds ---
../../out/201027/pca/transformer_encoder/raw/motor_toolkit.npy
../../out/201027/tsne/transformer_encoder/raw/motor_toolkit.npy
../../out/201027/pca/transformer_encoder/raw/pfamA_motors_balanced.npy
../../out/201027/tsne/transformer_encoder/raw/pfamA_motors_balanced.npy
../../out/201027/pca/transformer_encoder/raw/pfamA_random.npy
../../out/201027/tsne/transformer_encoder/raw/pfamA_random.npy
../../out/201027/pca/transformer_encoder/raw/pfamA_target.npy
../../out/201

In [11]:
# make one directory for each tuning method
import os, sys

# for model_dir in models_dir:
model_dir = "transformer_encoder_mlm/"
os.mkdir(pca_out_dir+model_dir)
os.mkdir(tsne_out_dir+model_dir)
for tuning_dir_ in tuning_dir:
    pca_path = pca_out_dir+model_dir+tuning_dir_
    print(pca_path)
    os.mkdir(pca_path)
    tsne_path = tsne_out_dir+model_dir+tuning_dir_
    os.mkdir(tsne_path);

../../out/201027/pca/transformer_encoder_mlm/evotune_balanced
../../out/201027/pca/transformer_encoder_mlm/evotune_balanced_target
../../out/201027/pca/transformer_encoder_mlm/mini_balanced
../../out/201027/pca/transformer_encoder_mlm/mini_balanced_target
../../out/201027/pca/transformer_encoder_mlm/raw


In [12]:
import glob
# for model in models_dir:
import time 

model = "transformer_encoder_mlm/"
print(model)
for tuning_dir_ in tuning_dir:
    print(tuning_dir_)
    dats = []
    for dat in data_dir:
        d_path = '../../out/201027/embedding/'+"transformer_encoder/"+dat+'_mlm_'+tuning_dir_ + '.npy'
        print(d_path)
        d = np.load(d_path)
#         if dat == 'motor_toolkit':
#             d = d[motor_toolkit_short_ind,:]
        if dat == 'pfamA_target':
            d = d[pfamA_target_sub_ind,:]
        dats.append(d)
    print(dats[0].shape)
    print(dats[1].shape)
    print(dats[2].shape)
    print(dats[3].shape)
    # first, embed each of the dataset themselves
    pca_res = []
    tsne_res = []
    for i, dat in enumerate(dats):
        print(data_dir[i])
        start_time = time.time()
        pca_curr = PCA(n_components=2).fit_transform(dat)
        pca_res.append(pca_curr)
        tsne_curr = PCA(n_components=5).fit_transform(dat)
        tsne_curr = TSNE(n_components=2).fit_transform(tsne_curr)
        tsne_res.append(tsne_curr)
        print("--- %s seconds ---" % (time.time() - start_time))
        
    for i,dat_name in enumerate(data_dir):
        pca_out_path = pca_out_dir+model+tuning_dir_+"/"+dat_name+".npy"
        tsne_out_path = tsne_out_dir+model+tuning_dir_+"/"+dat_name+".npy"
        print(pca_out_path)
        print(tsne_out_path)
        np.save(pca_out_path, pca_res[i])
        np.save(tsne_out_path, tsne_res[i])
    # then, perform several jointly embedding
    #1. target,random
    #2. balanced, random
    #3. toolkit,target
    for i,j in [(0,2),(1,2)]:
        start_time = time.time()
        target_tsne,background_tsne = joint_embed_tsne(dats[i],dats[j])
        target_pca,background_pca = joint_embed_pca(dats[i],dats[j])
        print("--- %s seconds ---" % (time.time() - start_time))
        pca_out_path_t = pca_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_target.npy"
        tsne_out_path_t = tsne_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_target.npy"
        pca_out_path_b = pca_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_background.npy"
        tsne_out_path_b = tsne_out_dir+model+tuning_dir_+"/"+data_dir[i]+"_"+data_dir[j]+"_background.npy"
        print(pca_out_path_t)
        print(pca_out_path_b)
        print(tsne_out_path_t)
        print(tsne_out_path_b)
        np.save(pca_out_path_t, target_pca)
        np.save(pca_out_path_b, background_pca)
        np.save(tsne_out_path_t, target_tsne)
        np.save(tsne_out_path_b, background_tsne)

transformer_encoder_mlm/
evotune_balanced
../../out/201027/embedding/transformer_encoder/motor_toolkit_mlm_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_motors_balanced_mlm_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_random_mlm_evotune_balanced.npy
../../out/201027/embedding/transformer_encoder/pfamA_target_mlm_evotune_balanced.npy
(3235, 12)
(18000, 12)
(1600, 12)
(5544, 12)
motor_toolkit
--- 52.56378650665283 seconds ---
pfamA_motors_balanced
--- 324.9635829925537 seconds ---
pfamA_random
--- 23.993781805038452 seconds ---
pfamA_target
--- 93.16517233848572 seconds ---
../../out/201027/pca/transformer_encoder_mlm/evotune_balanced/motor_toolkit.npy
../../out/201027/tsne/transformer_encoder_mlm/evotune_balanced/motor_toolkit.npy
../../out/201027/pca/transformer_encoder_mlm/evotune_balanced/pfamA_motors_balanced.npy
../../out/201027/tsne/transformer_encoder_mlm/evotune_balanced/pfamA_motors_balanced.npy
../../out/201027/pca/trans

--- 433.66476488113403 seconds ---
../../out/201027/pca/transformer_encoder_mlm/mini_balanced_target/pfamA_motors_balanced_pfamA_random_target.npy
../../out/201027/pca/transformer_encoder_mlm/mini_balanced_target/pfamA_motors_balanced_pfamA_random_background.npy
../../out/201027/tsne/transformer_encoder_mlm/mini_balanced_target/pfamA_motors_balanced_pfamA_random_target.npy
../../out/201027/tsne/transformer_encoder_mlm/mini_balanced_target/pfamA_motors_balanced_pfamA_random_background.npy
raw
../../out/201027/embedding/transformer_encoder/motor_toolkit_mlm_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_motors_balanced_mlm_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_random_mlm_raw.npy
../../out/201027/embedding/transformer_encoder/pfamA_target_mlm_raw.npy
(3235, 12)
(18000, 12)
(1600, 12)
(5544, 12)
motor_toolkit
--- 53.21200680732727 seconds ---
pfamA_motors_balanced
--- 326.4392113685608 seconds ---
pfamA_random
--- 23.892637014389038 seconds ---
pfamA_ta